# Poisson 2D

This solves:

$$ \Delta u = f \text{ in } \Omega = [0,1]^2$$ 

with

$$ f(x, y) = 10 \exp^{-50 [(x - 1/2)^2 + (y - 1/2)^2)]} $$

and boundary conditions given by

$$    u(x, y) = 0 \text{ for } x = 0 \text{ or } x = 1 $$
and

$$ \frac{\partial u}{\partial n}(x, y) = sin(5 x) \text{ for } y = 0 \text{ or } y = 1 $$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as pl
import numpy as np

from dolfin import *

parameters["form_compiler"]["cpp_optimize"] = False
parameters["form_compiler"]["optimize"] = False

def poisson(V):
    """ Solves Poisson's eq on the unit square.
    This code is based on DOLFIN's homonymous demo, (C) 2007-2011 Anders Logg

        V: FunctionSpace.
    """
    
    # Define Dirichlet boundary (x = 0 or x = 1)
    def boundary(x):
        return x[0] < DOLFIN_EPS or x[0] > 1.0 - DOLFIN_EPS

    # Define boundary condition
    u0 = project(Constant(0.0), V)  # Need to project to work around lack of evaluate_dofs() for Hermite
    bc = DirichletBC(V, u0, boundary)

    # Define variational problem
    u = TrialFunction(V)
    v = TestFunction(V)
    f = Expression("10*exp(-50*(pow(x[0] - 0.5, 2) + pow(x[1] - 0.5, 2)))", degree=2)
    g = Expression("sin(5*x[0])", degree=2)
    a = inner(grad(u), grad(v))*dx
    L = f*v*dx + g*v*ds

    # Compute solution
    u = Function(V)
    solve(a == L, u, bc)
    return u

In [ ]:
mesh = UnitSquareMesh(64, 64)
W = FunctionSpace(mesh, "Lagrange", 2)
lag = poisson(W)
plot(lag, cmap='bone')
_ = pl.title("Lagrange elements")

In [ ]:
V = FunctionSpace(mesh, "Hermite", 3)
her = poisson(V)
plot(her, cmap='bone')
_ = pl.title("Hermite elements")

In [ ]:
diff = project(lag-her, V)
diff.vector().abs()
plot(diff, cmap='bone')
_ = pl.title("Absolute difference")
print("Max. difference = %.4f" % diff.vector().max())

In [ ]:
xx = np.linspace(0,1,100)
pl.figure(figsize=(10,8))
error_threshold = 0.5
for y in np.linspace(0,1,4):
    pl.plot(xx, [min(error_threshold, np.abs(diff(x,y)/lag(x,y))) for x in xx],
            label="y = %.1f" % y)
    #pl.plot(xx, [diff(x,y) for x in xx], label="D@%.1f" % y)
    #pl.plot(xx,[her([x,y]) for x in xx], label="H@%.1f" % y)
    #pl.plot(xx,[lag([x,y]) for x in xx], label="L@%.1f" % y)
pl.title("Relative error (capped at %.1f) at multiple ordinates" % error_threshold)
_ = pl.legend()